In [1]:
from typing import Optional, Callable

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

In [10]:
meta_file = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/metadata.csv'
df_meta = pd.read_csv(meta_file)  # , index_col='cell_id'

In [11]:
df_meta

,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq
...,...,...,...,...,...
281523,96a60b026659,10,31800,hidden,multiome
281524,d493e546991e,10,31800,hidden,multiome
281525,05666c99aa48,10,31800,hidden,multiome
281526,121f946642b5,10,31800,hidden,multiome


In [14]:
x_data = pd.read_hdf('/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_inputs.h5', start=0, stop=50)

In [19]:
x_data.head()

gene_id,GL000194.1:114519-115365,GL000194.1:55758-56597,GL000194.1:58217-58957,GL000194.1:59535-60431,GL000195.1:119766-120427,GL000195.1:120736-121603,GL000195.1:137437-138345,GL000195.1:15901-16653,GL000195.1:22357-23209,GL000195.1:23751-24619,...,chrY:7722278-7723128,chrY:7723971-7724880,chrY:7729854-7730772,chrY:7731785-7732664,chrY:7810142-7811040,chrY:7814107-7815018,chrY:7818751-7819626,chrY:7836768-7837671,chrY:7869454-7870371,chrY:7873814-7874709
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.428336,0.0,0.0,0.0,0.0
fc0c60183c33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9b4a87e22ad0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
81cccad8cd81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15cb3d85c232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [22]:
x_data.shape

(50, 228942)

In [16]:
y_data = pd.read_hdf('/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_targets.h5', start=0, stop=50)

In [18]:
y_data.head()

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,4.893861,0.0,0.0,0.000000,0.0,5.583255,0.000000,4.893861
fc0c60183c33,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9b4a87e22ad0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,5.107832,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.107832
81cccad8cd81,0.0,4.507936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.195558,4.507936,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.195558
15cb3d85c232,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,5.531572,0.0,0.000000,4.842377,0.000000


In [26]:
y_data.shape[0]

50

In [33]:
y_data.loc['e31ca103a4ac'].values.shape

(23418,)

In [34]:
x_data.loc['e31ca103a4ac'].values.shape

(228942,)

------------------------------------------------------------------------------

In [2]:
class MyDataset(Dataset):
    def __init__(self,
                 features_file: str,
                 start_pos: int,
                 load_pos: int,
                 targets_file: Optional[str] = None,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None):
        self.features = pd.read_hdf(features_file, start=start_pos, stop=load_pos)
        if targets_file:
            self.targets = pd.read_hdf(targets_file, start=start_pos, stop=load_pos)
            assert self.features.shape[0] == self.targets.shape[0]
        else:
            self.targets = targets_file

        self.cell_ids = list(self.features.index)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.cell_ids)

    def __getitem__(self, item):
        cell_id = self.cell_ids[item]
        if self.targets is not None:
            x = self.features.loc[cell_id].values
            y = self.targets.loc[cell_id].values

            if self.transform:
                x = self.transform(x)

            if self.target_transform:
                y = self.target_transform(y)

            return x, y
        else:
            x = self.features.loc[cell_id].values
            if self.transform:
                x = self.transform(x)
            return x

In [3]:
X = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_inputs.h5'
Y = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_targets.h5'
Xt = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/test_multi_inputs.h5'

In [4]:
train_dataset = MyDataset(features_file=X, start_pos=0, load_pos=1000, targets_file=Y,
                          transform=torch.from_numpy, target_transform=torch.from_numpy)

test_dataset = MyDataset(features_file=Xt, start_pos=0, load_pos=1000, transform=torch.from_numpy)

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [6]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([2, 228942])
Labels batch shape: torch.Size([2, 23418])
